In [1]:
# Import the required libraries
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import os
import tqdm

In [2]:

# Set the CUDA_VISIBLE_DEVICES environment variable to specify GPU devices
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Use GPU devices 0 and 1

from flask import Flask, request, jsonify
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
# from torch import cuda
#from torch import cuda, LongTensor, FloatTensor, eq
#from transformers import (AutoModelForCausalLM, AutoTokenizer, AutoConfig, pipeline, 
#                          StoppingCriteria, StoppingCriteriaList)
# from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline)
import time
import re 


In [3]:
data_final_test= pd.read_csv("data_final_test.csv")
len(data_final_test)
# diagnosis_test= pd.read_csv("diagnosis_test.csv")
# len(diagnosis_test)
# treatment_given_test= pd.read_csv("treatment_given_test.csv")
# len(treatment_given_test)
# advice_on_discharge_test= pd.read_csv("advice_on_discharge_test.csv")
# len(advice_on_discharge_test)
# revisit_schedule_test= pd.read_csv("revisit_schedule_test.csv")
# len(revisit_schedule_test)
# revisit_investigation_test= pd.read_csv("revisit_investigation_test.csv")
# len(revisit_investigation_test)

800

In [4]:
data_final_test

,cr_no,discharge_summary,opip_notes,Ground_Truth,text,Result_Finetuned
0,189907.0,DIAGNOSIS:RECURRENT CARCINOMA OVARY TREATMENT...,recurrent ca ovary post multiple lines pet ct...,{'Diagnosis': 'RECURRENT CARCINOMA OVARY'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
1,210340.0,DIAGNOSIS: RECURRENT CARCINOMA OVARYTREATMENT ...,C/O RECC CA OVARY PATIENT REVIEWED NO FRESH CO...,{'Diagnosis': 'RECURRENT CARCINOMA OVARY'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
2,248604.0,DIAGNOSIS:Carcinoma Left Lateral Border Of To...,F/U/C OF VERRUCA VULGARIS ON PEMBROLIZUMAB IMP...,{'Diagnosis': 'VERRUCA VULGARIS'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
3,311049.0,DIAGNOSIS: Carcinoma Left Breast TREATMENT GI...,case reviewed PET s/o PR [provisional] No fres...,{'Diagnosis': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
4,133288.0,DIAGNOSIS:CA BREAST TREATMENT GIVEN: Inj. CYT...,Metastatic Ca Breast on chemotherapy issue: pu...,{'Diagnosis': 'Metastatic Ca Breast '},[INST]Answer the following question using only...,[INST]Answer the following question using only...
...,...,...,...,...,...,...
795,291196.0,DIAGNOSIS: Carcinoma Right Breast TREATMENT G...,AGRA DIAGNOSIS: CA (R) BREAST CLINICAL OBSERVA...,{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
796,327587.0,DIAGNOSIS:Carcinoma Right Breast TREATMENT GI...,Tab Pan 40 once a day x 7 days then SOS (BBF)...,"{'Revisit_Investigations': 'cbc lft kft , echo '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...
797,271953.0,DIAGNOSIS: Carcinoma Ovary TREATMENT GIVEN: I...,": DM, Hypertension DIAGNOSIS : ---------------...",{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...
798,331109.0,DIAGNOSIS: Metastatic Carcinoma Lung TREATMEN...,Rx: - inj zoldronat 3 mg in 100 ml ns over 30 ...,"{'Revisit_Investigations': 'cbc lft kft, '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...


In [5]:
# model_id="/datadrive1/RGCI/merged_LLaMa2_13B_Chat-finetuned_New_annotations_v10_vllm"
model_id="/datadrive1/llama/llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
# Load the model and the tokenizer. Set generation config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto")

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=300,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def solve_question(question_prompt):
    inputs = tokenizer(question_prompt, return_tensors="pt", padding=True, truncation=True,max_length= 2048).to("cuda")
    outputs = model.generate(**inputs, generation_config=generation_config)
    # except: 
    #     print(question_prompt)
    #     outputs= ""
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer

In [8]:
all_answers = []
val_data_prompts = list(data_final_test['text'])

for i in tqdm.tqdm(range(0, len(val_data_prompts), 1)):
    question_prompts = val_data_prompts[i:i+1]
    ans = solve_question(question_prompts)

    all_answers.extend(ans)
    torch.cuda.empty_cache()

100%|█████████████████████████████████████████| 800/800 [29:27<00:00,  2.21s/it]


In [9]:
all_answers

['[INST]Answer the following question using only the text delimited by triple backticks below. What is the patient’s cancer diagnosis? Include only the diagnosis and the laterality. Output the response in JSON format with a single key called \'Diagnosis\'. If you do not know the answer to the question, reply with \'Not Reported\'.``` recurrent ca ovary post multiple lines pet ct pd plan nab tortaxel advice admit daycare for inj nabtor taxel 100mg i/v in ward c1 d15 followup after 2 week```[/INST] Response:\n\n{\n"Diagnosis": {\n"Cancer": "Recurrent CA Ovary",\n"Laterality": "Not Reported"\n}\n}',
 '[INST]Answer the following question using only the text delimited by triple backticks below. What is the patient’s cancer diagnosis? Include only the diagnosis and the laterality. Output the response in JSON format with a single key called \'Diagnosis\'. If you do not know the answer to the question, reply with \'Not Reported\'.```C/O RECC CA OVARY PATIENT REVIEWED NO FRESH COMPLAINTS INV : 

In [10]:
len(all_answers)

800

In [13]:
# Result_Non_Finetuned
# Result_Finetuned

In [11]:
# data_final_test(V1): Finetuned time (85 Test samples):  40 min
# data_final_test(V1) : Non Finetuned time (85 Test samples): 10 min

# data_final_test(V1): Finetuned time (518 Test samples):  
# data_final_test(V1) : Non Finetuned time (518 Test samples): 1:42:14

# data_final_test: Finetuned time (v10_vllm) (800 Test samples):  21:32 min
# data_final_test : Non Finetuned time (800 Test samples): 29:27 min

# data_final_test['Result_Non_Finetuned'] = all_answers
# data_final_test.to_csv('data_final_test.csv', index=False)
# data_final_test

,cr_no,discharge_summary,opip_notes,Ground_Truth,text,Result_Finetuned,Result_Non_Finetuned
0,189907.0,DIAGNOSIS:RECURRENT CARCINOMA OVARY TREATMENT...,recurrent ca ovary post multiple lines pet ct...,{'Diagnosis': 'RECURRENT CARCINOMA OVARY'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
1,210340.0,DIAGNOSIS: RECURRENT CARCINOMA OVARYTREATMENT ...,C/O RECC CA OVARY PATIENT REVIEWED NO FRESH CO...,{'Diagnosis': 'RECURRENT CARCINOMA OVARY'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
2,248604.0,DIAGNOSIS:Carcinoma Left Lateral Border Of To...,F/U/C OF VERRUCA VULGARIS ON PEMBROLIZUMAB IMP...,{'Diagnosis': 'VERRUCA VULGARIS'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
3,311049.0,DIAGNOSIS: Carcinoma Left Breast TREATMENT GI...,case reviewed PET s/o PR [provisional] No fres...,{'Diagnosis': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
4,133288.0,DIAGNOSIS:CA BREAST TREATMENT GIVEN: Inj. CYT...,Metastatic Ca Breast on chemotherapy issue: pu...,{'Diagnosis': 'Metastatic Ca Breast '},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
...,...,...,...,...,...,...,...
795,291196.0,DIAGNOSIS: Carcinoma Right Breast TREATMENT G...,AGRA DIAGNOSIS: CA (R) BREAST CLINICAL OBSERVA...,{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
796,327587.0,DIAGNOSIS:Carcinoma Right Breast TREATMENT GI...,Tab Pan 40 once a day x 7 days then SOS (BBF)...,"{'Revisit_Investigations': 'cbc lft kft , echo '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
797,271953.0,DIAGNOSIS: Carcinoma Ovary TREATMENT GIVEN: I...,": DM, Hypertension DIAGNOSIS : ---------------...",{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
798,331109.0,DIAGNOSIS: Metastatic Carcinoma Lung TREATMEN...,Rx: - inj zoldronat 3 mg in 100 ml ns over 30 ...,"{'Revisit_Investigations': 'cbc lft kft, '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...


In [15]:
# Diagnosis : Finetuned time (25 Test samples):  12:37 min
# Diagnosis : Non Finetuned time (25 Test samples): 1:08 min

# Diagnosis : Finetuned time (90 Test samples):  18:11 min
# Diagnosis : Non Finetuned time (90 Test samples): 01:43 min

# Diagnosis : Finetuned time (v10_vllm) (150 Test samples): 01:36 min 
# Diagnosis : Non Finetuned time (150 Test samples): 03:24 min

# diagnosis_test['Result_Non_Finetuned'] = all_answers
# diagnosis_test.to_csv('diagnosis_test.csv', index=False)
# diagnosis_test

In [16]:
#Treatement Given: Finetuned time (149 Test samples): 39:33 min
#Treatment Given: Non Finetuned time (149 Test samples): 38:37 min

#Treatement Given: Finetuned time (179 Test samples): 1:19:11
#Treatment Given: Non Finetuned time (179 Test samples): 49:18

#Treatement Given: Finetuned chemo time (179 Test samples): 47:28 
#Treatment Given: Non Finetuned  chemo time (179 Test samples): 43:25

#Treatement Given: Finetuned time (69 Test samples): 07:39 min
#Treatment Given: Non Finetuned time (69 Test samples): 03:02 min

#Treatement Given: Finetuned (v10_vllm) time (113 Test samples): 03:53 min
#Treatment Given: Non Finetuned time (113 Test samples): 03:32 min

# treatment_given_test['Result_Non_Finetuned'] = all_answers
# treatment_given_test.to_csv('treatment_given_test.csv', index=False)
# treatment_given_test

In [17]:
# Advice At Discharge : Finetuned time (156 Test samples): 1:14:36 hours
# Advice At Discharge : Non Finetuned time (156 Test samples): 39:47 min

# Advice At Discharge : Finetuned time (156 Test samples): 1:14:36 hours
# Advice At Discharge : Non Finetuned time (156 Test samples): 39:47 min

# Advice At Discharge : Finetuned time (214 Test samples): 1:37:14 
# Advice At Discharge : Non Finetuned time (214 Test samples): 59:13 min

# Advice At Discharge : Finetuned time chemo (69 Test samples): 08:05
# Advice At Discharge : Non Finetuned time chemo (69 Test samples): 02:38

# Advice At Discharge : Finetuned (v10_vllm) time chemo (113 Test samples): 02:58 min
# Advice At Discharge : Non Finetuned time chemo (113 Test samples): 04:43 min

# advice_on_discharge_test['Result_Non_Finetuned'] = all_answers
# advice_on_discharge_test.to_csv('advice_on_discharge_test.csv', index=False)
# print(len(advice_on_discharge_test))
# advice_on_discharge_test

In [18]:
# Revist Schedule : Finetuned time (30 Test samples): 15:31 min
# Revist Schedule : Non Finetuned time (30 Test samples): 02:39 min

# Revist Schedule : Finetuned time (145 Test samples):  28:39 min
# Revist Schedule : Non Finetuned time (145 Test samples): 06:20 min

# Revist Schedule : Finetuned time (145 Test samples):   23:49 min
# Revist Schedule : Non Finetuned time (145 Test samples):  06:42 min

# Revist Schedule : Finetuned (v10_vllm) time (212 Test samples): 10:04 min  
# Revist Schedule : Non Finetuned time (212 Test samples): 09:21 min


# revisit_schedule_test['Result_Non_Finetuned'] = all_answers
# revisit_schedule_test.to_csv('revisit_schedule_test.csv', index=False)
# print(len(revisit_schedule_test))
# revisit_schedule_test

In [19]:
# Revist Investigation : Finetuned time (30 Test samples): 14:48 min
# Revist Investigation : Non Finetuned time (30 Test samples): 02:05 min

# Revist Investigation : Finetuned time (145 Test samples): 09:55 min
# Revist Investigation : Non Finetuned time (145 Test samples): 05:20 min


# Revist Investigation : Finetuned (v10_vllm)  time (212 Test samples): 02:20 min
# Revist Investigation : Non Finetuned time (212 Test samples): 07:56 min


# revisit_investigation_test['Result_Non_Finetuned'] = all_answers
# revisit_investigation_test.to_csv('revisit_investigation_test.csv', index=False)
# print(len(revisit_investigation_test))
# revisit_investigation_test

212


,cr_no,discharge_summary,opip_notes,revisit_investigations,text,Result_Finetuned,Result_Non_Finetuned
0,298225.0,DIAGNOSIS:Carcinoma Lung &ndash;Adenocarcinom...,Adv : - Inj zoldasta 4 mg in 100 ml NS over 30...,"{'Revisit_Investigations': ' CBC, KFT, LFT, Ca...",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
1,287317.0,DIAGNOSIS:Recurrent Carcinoma Esophagus TREAT...,To procure outside blocks (adrenal lesion) In...,"{'Revisit_Investigations': ' CBC LFT KFT , CAL...",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
2,334670.0,DIAGNOSIS:NSCLC - NOS TREATMENT GIVEN: Inj. P...,Inj Grastim (Vial) 300mcg S/C once a day x 2 ...,{'Revisit_Investigations': 'CBC KFT LFT Calciu...,[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
3,339585.0,DIAGNOSIS: Ca Base of Tongue - cT4aN2cM0 ...,Adv : Candid mouth paint local applicationTDS ...,"{'Revisit_Investigations': 'CBC,KFT'}",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
4,231017.0,DIAGNOSIS: Carcinoma Left Breast TREATMENT GI...,DIAGNOSIS: ----------------- CA BREAST (R) CL...,{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
...,...,...,...,...,...,...,...
207,291196.0,DIAGNOSIS: Carcinoma Right Breast TREATMENT G...,AGRA DIAGNOSIS: CA (R) BREAST CLINICAL OBSERVA...,{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
208,327587.0,DIAGNOSIS:Carcinoma Right Breast TREATMENT GI...,Tab Pan 40 once a day x 7 days then SOS (BBF)...,"{'Revisit_Investigations': 'cbc lft kft , echo '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
209,271953.0,DIAGNOSIS: Carcinoma Ovary TREATMENT GIVEN: I...,": DM, Hypertension DIAGNOSIS : ---------------...",{'Revisit_Investigations': 'Not Reported'},[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
210,331109.0,DIAGNOSIS: Metastatic Carcinoma Lung TREATMEN...,Rx: - inj zoldronat 3 mg in 100 ml ns over 30 ...,"{'Revisit_Investigations': 'cbc lft kft, '}",[INST]Answer the following question using only...,[INST]Answer the following question using only...,[INST]Answer the following question using only...
